In [1]:
with open('/content/drive/MyDrive/Colab Notebooks/NLP/Korean_movie_reviews_2016.txt', encoding='utf-8') as f:
    docs = [doc.strip().split('\t') for doc in f]
    docs = [(doc[0], int(doc[1])) for doc in docs if len(doc) == 2]
    texts, labels = zip(*docs)

words_list = [doc.strip().split() for doc in texts]
print(words_list[:2])
print(labels[:2])
print(len(texts))
print(len(labels))

[['부산', '행', '때문', '너무', '기대하고', '봤'], ['한국', '좀비', '영화', '어색하지', '않게', '만들어졌', '놀랍']]
(0, 1)
165384
165384


In [2]:
from tensorflow.keras.utils import to_categorical
y_one_hot = to_categorical(labels)

In [6]:
import numpy as np
class_counts = np.sum(y_one_hot, axis=0)
class_counts

array([78578., 86806.], dtype=float32)

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(texts, y_one_hot, test_size=0.2, random_state=0)

In [8]:
len(X_train), len(X_test), len(y_train), len(y_test)

(132307, 33077, 132307, 33077)

In [13]:
class_counts_train = np.sum(y_train, axis=0)
print(class_counts_train)
class_counts_test = np.sum(y_test, axis=0)
print(class_counts_test)

[62909. 69398.]
[15669. 17408.]


In [5]:
X_train[0], y_train[0]

('정말 이런 영화 보는 시간 넘아 깝 고통스러웠 영화 보고 아팠', array([1., 0.], dtype=float32))

In [14]:
from transformers import BertTokenizer, TFAlbertForSequenceClassification
tokenizer= BertTokenizer.from_pretrained("kykim/albert-kor-base")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'AlbertTokenizer'. 
The class this function is called from is 'BertTokenizer'.


In [15]:
tokenizer.model_input_names

['input_ids', 'token_type_ids', 'attention_mask']

In [16]:
X_train_tokenized = tokenizer(X_train, return_tensors="np", max_length=30, padding='max_length', truncation=True)
X_test_tokenized = tokenizer(X_test, return_tensors="np", max_length=30, padding='max_length', truncation=True)

In [17]:
X_train_tokenized

{'input_ids': array([[    2, 14038, 14119, ...,     0,     0,     0],
       [    2, 15024,  3363, ...,     0,     0,     0],
       [    2, 14038,  3424, ...,     0,     0,     0],
       ...,
       [    2, 14241, 14632, ...,     0,     0,     0],
       [    2, 14119, 16897, ...,     0,     0,     0],
       [    2, 14632, 14010, ...,     0,     0,     0]]), 'token_type_ids': array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]]), 'attention_mask': array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0]])}

In [18]:
X_train_tokenized.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [19]:
X_train_tokenized['input_ids'].shape

(132307, 30)

In [20]:
type(X_train_tokenized['input_ids']), type(y_train)

(numpy.ndarray, numpy.ndarray)

In [21]:
X_train_data = {'input_ids':X_train_tokenized['input_ids'],
       'attention_mask':X_train_tokenized['attention_mask']},

In [22]:
X_test_data = {'input_ids':X_test_tokenized['input_ids'],
       'attention_mask':X_test_tokenized['attention_mask']},

In [23]:
y_test

array([[0., 1.],
       [1., 0.],
       [1., 0.],
       ...,
       [1., 0.],
       [1., 0.],
       [1., 0.]], dtype=float32)

In [24]:
#Tensorflow based!!
albert_model = TFAlbertForSequenceClassification.from_pretrained("kykim/albert-kor-base", num_labels=2, from_pt=True)
albert_model.summary()

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFAlbertForSequenceClassification: ['albert.embeddings.position_ids', 'sop_classifier.classifier.bias', 'sop_classifier.classifier.weight']
- This IS expected if you are initializing TFAlbertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFAlbertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFAlbertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions a

Model: "tf_albert_for_sequence_classification"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 albert (TFAlbertMainLayer)  multiple                  13186816  
                                                                 
 dropout_4 (Dropout)         multiple                  0         
                                                                 
 classifier (Dense)          multiple                  1538      
                                                                 
Total params: 13188354 (50.31 MB)
Trainable params: 13188354 (50.31 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [25]:
for l in albert_model.layers:
  print(l)

In [ ]:
pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.6 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 2

In [ ]:
'''import datasets
from datasets import Dataset, DatasetDict
print('to_tf_dataset' in dir(Dataset))'''

True


In [ ]:
'''train_dataset'''

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'label'],
    num_rows: 132307
})

In [26]:
dir(albert_model)

['_SCALAR_UPRANKING_ON',
 '_TF_MODULE_IGNORED_PROPERTIES',
 '__annotations__',
 '__call__',
 '__class__',
 '__copy__',
 '__deepcopy__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_activity_regularizer',
 '_add_trackable',
 '_add_trackable_child',
 '_add_variable_with_custom_getter',
 '_aggregate_exact_metrics',
 '_assert_compile_was_called',
 '_assert_weights_created',
 '_auto_class',
 '_auto_get_config',
 '_auto_track_sub_layers',
 '_autocast',
 '_autographed_call',
 '_autotune_steps_per_execution',
 '_base_model_initialized',
 '_build_input_shape',
 '_call_spec',
 '_callable_losses',
 '_captured_weight_regularizer',
 '_cast_single_input',
 '

In [28]:
albert_model.compile()

In [29]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy

#optimizer = Adam(learning_rate=5e-5)
#loss = BinaryCrossentropy()
albert_model.compile(
    loss = BinaryCrossentropy(),
    metrics = ['accuracy'])

In [12]:
albert_model.optimizer

In [31]:
import numpy as np
import tensorflow.keras

In [32]:
history = albert_model.fit(
    x = X_train_data,
    y = y_train,
    epochs=2) #batch size 32

Epoch 1/2


Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
4135/4135 [==============================] - 814s 185ms/step - loss: 7.2910 - accuracy: 0.5245
Epoch 2/2
4135/4135 [==============================] - 762s 184ms/step - loss: 7.2925 - accuracy: 0.5245


In [59]:
!pip install transformers==4.18.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 35.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 48.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 59.3 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.41.2
    Uninstalling transformers-4.41.2:
      Successfully uninstalled transformers-4.41.2


In [1]:
from transformers import TFTrainer

In [33]:
albert_model.save('saved_model/my_albert_model')

In [46]:
albert_model.evaluate(X_test_data, np.array(y_test))

1034/1034 [==============================] - 98s 80ms/step - loss: 7.2237 - accuracy: 1.0000


[7.2237162590026855, 1.0]

In [52]:
y_preds = albert_model.predict(X_test_data)

1034/1034 [==============================] - 80s 77ms/step


In [57]:
y_preds.logits[100]

array([-0.64541286,  6.0648246 ], dtype=float32)

In [45]:
import tensorflow as tf
prediction_probs = tf.nn.softmax(y_preds.logits,axis=0).numpy()

In [46]:
prediction_probs

array([[3.0232477e-05, 3.0232466e-05],
       [3.0232477e-05, 3.0232466e-05],
       [3.0232482e-05, 3.0232452e-05],
       ...,
       [3.0232482e-05, 3.0232466e-05],
       [3.0232484e-05, 3.0232466e-05],
       [3.0232484e-05, 3.0232466e-05]], dtype=float32)

In [47]:
y_predictions = np.argmax(prediction_probs, axis=1)

In [48]:
from collections import Counter
y_predictions_count = Counter(y_predictions)
print(y_predictions_count)

Counter({0: 32879, 1: 198})


In [53]:
import tensorflow as tf
#y_preds = albert_model.predict(dict(X_test_tokenized))
#prediction_probs = tf.nn.softmax(y_preds.logits,axis=1).numpy()
#y_predictions = np.argmax(prediction_probs, axis=1)
#y_test = np.argmax(y_test, axis=1)
from sklearn.metrics import classification_report
print(classification_report(y_predictions, y_test))

  23/1034 [..............................] - ETA: 1:32

KeyboardInterrupt: 

In [55]:
from sklearn.metrics import classification_report
print(classification_report(y_predictions, y_test))

              precision    recall  f1-score   support

           0       1.00      0.47      0.64     33077
           1       0.00      0.00      0.00         0

    accuracy                           0.47     33077
   macro avg       0.50      0.24      0.32     33077
weighted avg       1.00      0.47      0.64     33077



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
